In [34]:
import os
import re
import random
from math import sqrt
import json
import numpy as np
import pandas as pd
import lightgbm as lgb
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.metrics import cohen_kappa_score, make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.stats import ks_2samp

pd.show_versions()


INSTALLED VERSIONS
------------------
commit           : None
python           : 3.7.1.final.0
python-bits      : 64
OS               : Windows
OS-release       : 7
machine          : AMD64
processor        : Intel64 Family 6 Model 58 Stepping 9, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : None.None

pandas           : 0.25.3
numpy            : 1.17.4
pytz             : 2019.3
dateutil         : 2.8.1
pip              : 19.3.1
setuptools       : 39.0.1
Cython           : None
pytest           : None
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : None
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 2.10.3
IPython          : 7.10.2
pandas_datareader: None
bs4              : None
bottleneck       : None
fastparquet      : None
gcsfs            : None
lxml.etree       : None
matplotlib       : 3

In [2]:
def _log(str):
    os.system(f'echo \"{str}\"')
    print(str)

In [3]:
NAN = '__NAN__'
INPUT_ROOT = '../input/data-science-bowl-2019'
JOIN_KEY = ['installation_id', 'game_session', 'title']
TARGET = 'accuracy_group'
FEATURES = {
    'event_id', 
    'game_session', 
    'timestamp', 
    'installation_id', 
    'event_count',
    'event_code', 
    'game_time', 
    'title', 
    'type', 
    'world',
    'event_data'
}
EVENT_CODES = ['2000', '2010', '2020', '2025', '2030', '2035', '2040', '2050', '2060', '2070', '2075', '2080', '2081', '2083', '3010', '3020', '3021', '3110', '3120', '3121', '4010', '4020', '4021', '4022', '4025', '4030', '4031', '4035', '4040', '4045', '4050', '4070', '4080', '4090', '4095', '4100', '4110', '4220', '4230', '4235', '5000', '5010']
SEED = 31
FOLDS = 3
ESTIMATORS = 1000

In [4]:
def _init():
    # Characters such as empty strings '' or numpy.inf are considered NA values
    pd.set_option('use_inf_as_na', True)
    pd.set_option('display.max_columns', 999)
    pd.set_option('display.max_rows', 999)
    
    
_init()

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


seed_everything(SEED)

In [6]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk(INPUT_ROOT):
    for filename in filenames:
        _log(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

../input/data-science-bowl-2019\sample_submission.csv
../input/data-science-bowl-2019\test.csv
../input/data-science-bowl-2019\test.csv.zip
../input/data-science-bowl-2019\train.csv
../input/data-science-bowl-2019\train.csv.zip
../input/data-science-bowl-2019\train_labels.csv
../input/data-science-bowl-2019\train_labels.csv.zip


In [7]:
%%time
train_raw = pd.read_csv(f'{INPUT_ROOT}/train.csv', usecols=FEATURES)
train_labels = pd.read_csv(f'{INPUT_ROOT}/train_labels.csv', usecols=JOIN_KEY + [TARGET])
test_raw = pd.read_csv(f'{INPUT_ROOT}/test.csv', usecols=FEATURES)
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17690 entries, 0 to 17689
Data columns (total 4 columns):
game_session       17690 non-null object
installation_id    17690 non-null object
title              17690 non-null object
accuracy_group     17690 non-null int64
dtypes: int64(1), object(3)
memory usage: 552.9+ KB
Wall time: 54.8 s


# Add labels to train data

In [8]:
def _remove_unlabelled_data(train_raw, train_labels):
    return train_raw[train_raw['installation_id'].isin(train_labels['installation_id'].unique())]


train_raw = _remove_unlabelled_data(train_raw, train_labels)

In [9]:
%%time
def _add_labels(train_raw, train_labels, on):
    return pd.merge(train_raw, train_labels, on=on, how='left')


train_raw = _add_labels(train_raw, train_labels, on=JOIN_KEY)
del train_labels

Wall time: 4.7 s


# Extract event data JSON

In [10]:
def _concat_columns(df1, df2):
    """Concatenate the columns of two pandas dataframes in the order of the operands.
    Both dataframes must have the same number of rows.
    """
    assert len(df1) == len(df2)
    res = pd.concat([df1, df2.reindex(df1.index)], axis=1, join='inner')
    assert len(res) == len(df1)
    return res
    

def _extract_event_data(df, keep_cols, chunk_size=1000000):
    res = pd.DataFrame()
    _len = len(df)
    for i in tqdm(range(0, _len, chunk_size)):
        if i + chunk_size < _len:
            chunk = df[i:i + chunk_size].copy()
        else:
            chunk = df[i:].copy()
        ed = pd.io.json.json_normalize(chunk['event_data'].apply(json.loads)).add_prefix('ed.')
        ed = ed[keep_cols]
        chunk = _concat_columns(chunk, ed)
        res = pd.concat([res, chunk], ignore_index=True, sort=False)
    assert len(df) == len(res)
    return res


keep_cols = ['ed.identifier', 'ed.duration', 'ed.level', 'ed.round', 'ed.correct', 'ed.misses',
            'ed.weight', 'ed.total_duration', 'ed.source']
train_raw = _extract_event_data(train_raw, keep_cols)
test_raw = _extract_event_data(test_raw, keep_cols)

100%|████████████████████████████████████████████████████| 2/2 [01:38<00:00, 49.38s/it]


In [11]:
test_raw.info(max_cols=999)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156414 entries, 0 to 1156413
Data columns (total 20 columns):
event_id             1156414 non-null object
game_session         1156414 non-null object
timestamp            1156414 non-null object
event_data           1156414 non-null object
installation_id      1156414 non-null object
event_count          1156414 non-null int64
event_code           1156414 non-null int64
game_time            1156414 non-null int64
title                1156414 non-null object
type                 1156414 non-null object
world                1156414 non-null object
ed.identifier        337810 non-null object
ed.duration          336619 non-null float64
ed.level             84208 non-null float64
ed.round             598718 non-null float64
ed.correct           69806 non-null object
ed.misses            23658 non-null float64
ed.weight            50353 non-null float64
ed.total_duration    170228 non-null float64
ed.source            144623 non-null obje

In [12]:
train_raw.info(max_cols=999)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7734558 entries, 0 to 7734557
Data columns (total 21 columns):
event_id             object
game_session         object
timestamp            object
event_data           object
installation_id      object
event_count          int64
event_code           int64
game_time            int64
title                object
type                 object
world                object
accuracy_group       float64
ed.identifier        object
ed.duration          float64
ed.level             float64
ed.round             float64
ed.correct           object
ed.misses            float64
ed.weight            float64
ed.total_duration    float64
ed.source            object
dtypes: float64(7), int64(3), object(11)
memory usage: 1.2+ GB


In [13]:
# All event ids in test set also exist in train set
test_set = set(test_raw['event_id'])
train_set = set(train_raw['event_id'])
vs = test_set - train_set
_log(f'{len(vs)} event_ids exist in test set but not train set.')

0 event_ids exist in test set but not train set.


In [14]:
EVENT_IDS = sorted(test_raw['event_id'].unique())
_log(f'{len(EVENT_IDS)} EVENT_IDS={EVENT_IDS}')

365 EVENT_IDS=['0086365d', '00c73085', '022b4259', '02a42007', '0330ab6a', '0413e89d', '04df9b66', '05ad839b', '06372577', '070a5291', '08fd73f3', '08ff79ad', '0a08139c', '0d18d96c', '0d1da71f', '0db6d71d', '1325467d', '1340b8d7', '1375ccb7', '14de4c5d', '155f62a4', '1575e76c', '15a43e5b', '15ba1109', '15eb4a7d', '15f99afc', '160654fd', '16667cc5', '16dffff1', '17113b36', '19967db1', '1996c610', '1af8be29', '1bb5fbdb', '1beb320a', '1c178d24', '1cc7cfca', '1cf54632', '1f19558b', '222660ff', '2230fab4', '250513af', '25fa8af4', '262136f4', '26a5a3dd', '26fd2d99', '27253bdc', '28520915', '28a4eb9a', '28ed704e', '28f975ea', '29bdd9ba', '29f54413', '2a444e03', '2a512369', '2b058fe3', '2b9272f4', '2c4e6db0', '2dc29e21', '2dcad279', '2fb91ec1', '30614231', '30df3273', '31973d56', '3323d7e9', '33505eae', '3393b68b', '363c86c9', '363d3849', '36fa3ebe', '37937459', '37c53127', '37db1c2f', '37ee8496', '38074c54', '392e14df', '3a4be871', '3afb49e6', '3afde5dd', '3b2048ee', '3babcb9b', '3bb91ced', '

In [15]:
TITLES = test_raw['title'].unique()
test_raw['title'].value_counts()

Bottle Filler (Activity)         112223
Scrub-A-Dub                       96619
Chow Time                         93142
Sandcastle Builder (Activity)     82132
Fireworks (Activity)              61032
Bug Measurer (Activity)           59886
Bubble Bath                       55264
Dino Drink                        53989
Dino Dive                         49368
Crystals Rule                     46867
Chicken Balancer (Activity)       45874
All Star Sorting                  45863
Flower Waterer (Activity)         43819
Happy Camel                       39806
Pan Balance                       38649
Watering Hole (Activity)          35367
Air Show                          27119
Egg Dropper (Activity)            25941
Leaf Leader                       25574
Cauldron Filler (Assessment)      23440
Mushroom Sorter (Assessment)      21962
Bird Measurer (Assessment)        20086
Cart Balancer (Assessment)        19235
Chest Sorter (Assessment)         17904
Crystal Caves - Level 2            1450


In [16]:
TYPES = test_raw['type'].unique()
test_raw['type'].value_counts()

Game          572260
Activity      466274
Assessment    102627
Clip           15253
Name: type, dtype: int64

In [17]:
WORLDS = test_raw['world'].unique()
test_raw['world'].value_counts()

MAGMAPEAK       511291
TREETOPCITY     332295
CRYSTALCAVES    311387
NONE              1441
Name: world, dtype: int64

In [18]:
test_raw['ed.source'].value_counts()

resources    64271
scale         9336
right         8586
left          7890
7             6791
9             5974
8             5581
10            5106
11            4514
2             4124
5             3457
12            3376
4             3186
1             2816
N/A           2770
6             2045
3             2045
middle        1144
Heaviest       641
Heavy          552
Lightest       418
Name: ed.source, dtype: int64

In [19]:
test_raw['ed.identifier'].value_counts()

1438                                                                                                                                                                                                                                                                                        14812
1437                                                                                                                                                                                                                                                                                         6738
Dot_SoCool                                                                                                                                                                                                                                                                                   6594
507                                                                                                                               

In [20]:
vs = sorted(train_raw['type'].unique())
_log(f'{len(vs)} train_raw type={vs}')

4 train_raw type=['Activity', 'Assessment', 'Clip', 'Game']


In [21]:
vs = sorted(train_raw['world'].unique())
_log(f'{len(vs)} train_raw type={vs}')

4 train_raw type=['CRYSTALCAVES', 'MAGMAPEAK', 'NONE', 'TREETOPCITY']


In [22]:
vs = sorted(train_raw['event_code'].unique())
_log(f'{len(vs)} train_raw type={vs}')

42 train_raw type=[2000, 2010, 2020, 2025, 2030, 2035, 2040, 2050, 2060, 2070, 2075, 2080, 2081, 2083, 3010, 3020, 3021, 3110, 3120, 3121, 4010, 4020, 4021, 4022, 4025, 4030, 4031, 4035, 4040, 4045, 4050, 4070, 4080, 4090, 4095, 4100, 4110, 4220, 4230, 4235, 5000, 5010]


In [23]:
vs = sorted(train_raw['title'].unique())
_log(f'{len(vs)} train_raw titles={vs}')

44 train_raw titles=['12 Monkeys', 'Air Show', 'All Star Sorting', 'Balancing Act', 'Bird Measurer (Assessment)', 'Bottle Filler (Activity)', 'Bubble Bath', 'Bug Measurer (Activity)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Chicken Balancer (Activity)', 'Chow Time', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Crystals Rule', 'Dino Dive', 'Dino Drink', 'Egg Dropper (Activity)', 'Fireworks (Activity)', 'Flower Waterer (Activity)', 'Happy Camel', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Leaf Leader', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', 'Pan Balance', "Pirate's Tale", 'Rulers', 'Sandcastle Builder (Activity)', 'Scrub-A-Dub', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Watering Hole (Activity)', 'Welcome to Lost Lagoon!']


In [24]:
vs = sorted(test_raw['title'].unique())
_log(f'{len(vs)} test titles={vs}')

44 test titles=['12 Monkeys', 'Air Show', 'All Star Sorting', 'Balancing Act', 'Bird Measurer (Assessment)', 'Bottle Filler (Activity)', 'Bubble Bath', 'Bug Measurer (Activity)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Chicken Balancer (Activity)', 'Chow Time', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Crystals Rule', 'Dino Dive', 'Dino Drink', 'Egg Dropper (Activity)', 'Fireworks (Activity)', 'Flower Waterer (Activity)', 'Happy Camel', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Leaf Leader', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', 'Pan Balance', "Pirate's Tale", 'Rulers', 'Sandcastle Builder (Activity)', 'Scrub-A-Dub', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Watering Hole (Activity)', 'Welcome to Lost Lagoon!']


In [25]:
def _transform_timestamp(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df


train_raw = _transform_timestamp(train_raw)
test_raw = _transform_timestamp(test_raw)

In [26]:
%%time
def _set_string_type(df, cols):
    df[cols] = df[cols].fillna(NAN).astype(str)
    return df


cols = ['event_code', 'timestamp', 'ed.identifier', 'ed.source']
train_raw = _set_string_type(train_raw, cols=cols)
test_raw = _set_string_type(test_raw, cols=cols)

Wall time: 1min 28s


In [27]:
%%time
def _sort_it(df):
    return df.sort_values(by=['installation_id', 'timestamp'])


train_raw = _sort_it(train_raw)
test_raw = _sort_it(test_raw)

Wall time: 26.9 s


# Multiple accuracy groups per installation id
In the train set, there are multiple accuracy groups per installation id. The task is to predict the accuracy group of the **last** assessment for a given installation id.

In [28]:
vs = train_raw[train_raw[TARGET].notna()].groupby('installation_id')[TARGET].nunique()
vs

installation_id
0006a69f    3
0006c192    3
00129856    1
001d0ed0    3
00225f67    1
           ..
ff9305d7    2
ff9715db    4
ffc90c32    2
ffd2871d    1
ffeb0b1b    2
Name: accuracy_group, Length: 3614, dtype: int64

In [29]:
def _log_smoothing(df, cols):
    for col in cols:
        df[col] = np.log(df[col] + 1)
    return df


#cols = ['event_count', 'game_time']
#train = _log_smoothing(train, cols)
#test = _log_smoothing(test, cols)

# Train-test split not by time
Both train and test sets span the same time period.

In [30]:
_log(f'train_raw[timestamp] is from {train_raw.timestamp.min()} to {train_raw.timestamp.max()}')
_log(f'test_raw[timestamp] is from {test_raw.timestamp.min()} to {test_raw.timestamp.max()}')

train_raw[timestamp] is from 2019-07-23 14:38:25.256000+00:00 to 2019-10-14 21:57:26.930000+00:00
test_raw[timestamp] is from 2019-07-24 00:04:25.361000+00:00 to 2019-10-14 21:00:34.858000+00:00


In [31]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7734558 entries, 0 to 7734557
Data columns (total 21 columns):
event_id             object
game_session         object
timestamp            object
event_data           object
installation_id      object
event_count          int64
event_code           object
game_time            int64
title                object
type                 object
world                object
accuracy_group       float64
ed.identifier        object
ed.duration          float64
ed.level             float64
ed.round             float64
ed.correct           object
ed.misses            float64
ed.weight            float64
ed.total_duration    float64
ed.source            object
dtypes: float64(7), int64(2), object(12)
memory usage: 1.3+ GB


# Feature Engineering

In [69]:
def _key(s):
    return re.sub(r'[\W\s]', '', s).lower()


def _cutoff(df, TARGET):
    return df[df[TARGET].notna()]['timestamp'].max()


def _trim_events_after_last_assessment(df, cutoff):
    res = df[df['timestamp'] <= cutoff]
    #_log(f'cutoff: before={df.shape}, after={res.shape}')
    return res
    
    
def _target_variable(df, cutoff, TARGET):
    vs = df[df['timestamp'] == cutoff][TARGET].values
    assert len(set(vs)) == 1
    return int(float(vs[0]))
    
    
def _game_session_stats(df, col, suffix):
    res = {}
    groups = [
        ['game_session'],
        ['game_session', 'type'],
        ['game_session', 'world'],
        ['game_session', 'title']
    ]
    for g in groups:
        vs = df.groupby(g)[col].transform('max')
        qs = vs.quantile([0.25, 0.5, 0.75], interpolation='lower').to_numpy()
        k = _key(''.join(g))
        res[f'{col}_p50_{k}{suffix}'] = np.int32([qs[1]])
        res[f'{col}_max_{k}{suffix}'] = np.int32([vs.max()])
        res[f'{col}_std_{k}{suffix}'] = np.float32([vs.std(ddof=0)])
        res[f'{col}_iqr_{k}{suffix}'] = np.int32([qs[2] - qs[0]])
    return res


def _event_id_features(df, EVENT_IDS, TITLES, TYPES, WORLDS, suffix):
    res = {}
    for eid in EVENT_IDS:
        res[f'eid_{eid}{suffix}'] = np.int32([sum(df['event_id'] == eid)])
        for t in TYPES:
            res[f'eid_{eid}_{_key(t)}{suffix}'] = np.int32([sum((df['event_id'] == eid) & (df['type'] == t))])
            
        for t in WORLDS:
            res[f'eid_{eid}_{_key(t)}{suffix}'] = np.int32([sum((df['event_id'] == eid) & (df['world'] == t))])
            
        for t in TITLES:
            res[f'eid_{eid}_{_key(t)}{suffix}'] = np.int32([sum((df['event_id'] == eid) & (df['title'] == t))])
        
    return res


def _event_code_features(df, EVENT_CODES, TITLES, TYPES, WORLDS, suffix):
    res = {}
    for code in EVENT_CODES:
        res[f'event_{code}{suffix}'] = np.int32([sum(df['event_code'] == code)])
        for t in TYPES:
            res[f'event_{code}_{_key(t)}{suffix}'] = np.int32([sum((df['event_code'] == code) & (df['type'] == t))])
            
        for t in WORLDS:
            res[f'event_{code}_{_key(t)}{suffix}'] = np.int32([sum((df['event_code'] == code) & (df['world'] == t))])
            
        for t in TITLES:
            res[f'event_{code}_{_key(t)}{suffix}'] = np.int32([sum((df['event_code'] == code) & (df['title'] == t))])
        
    return res


def _event_data_features(df, suffix):
    res = {}
    res[f'ed_duration{suffix}'] = np.int32(df['ed.duration'].fillna(0).max())
    res[f'ed_total_duration{suffix}'] = np.int32(df['ed.total_duration'].fillna(0).max())
    res[f'ed_level{suffix}'] = np.int32(df['ed.level'].fillna(0).max())
    res[f'ed_round{suffix}'] = np.int32(df['ed.round'].fillna(0).max())
    res[f'ed_correct{suffix}'] = np.int32(df['ed.correct'].fillna(0).max())
    res[f'ed_misses{suffix}'] = np.int32(df['ed.misses'].fillna(0).max())
    res[f'ed_weight{suffix}'] = np.int32(df['ed.weight'].fillna(0).max())
    res[f'ed_source_resources{suffix}'] = np.int32([sum(df['ed.source'] == 'resources')])
    res[f'ed_source_right{suffix}'] = np.int32([sum(df['ed.source'] == 'right')])
    res[f'ed_source_left{suffix}'] = np.int32([sum(df['ed.source'] == 'left')])
    res[f'ed_source_scale{suffix}'] = np.int32([sum(df['ed.source'] == 'scale')])
    res[f'ed_source_middle{suffix}'] = np.int32([sum(df['ed.source'] == 'middle')])
    res[f'ed_source_heaviest{suffix}'] = np.int32([sum(df['ed.source'] == 'Heaviest')])
    res[f'ed_source_heavy{suffix}'] = np.int32([sum(df['ed.source'] == 'Heavy')])
    res[f'ed_source_lightest{suffix}'] = np.int32([sum(df['ed.source'] == 'Lightest')])
    n = 0
    for i in range(1, 13):
        n += sum(df['ed.source'] == str(i))
    res[f'ed_source_numbered{suffix}'] = np.int32([n])
    res[f'ed_id_dot{suffix}'] = np.int32([sum(df['ed.identifier'].str.contains('Dot_', regex=False))])
    res[f'ed_id_buddy{suffix}'] = np.int32([sum(df['ed.identifier'].str.contains('Buddy_', regex=False))])
    res[f'ed_id_cleo{suffix}'] = np.int32([sum(df['ed.identifier'].str.contains('Cleo_', regex=False))])
    res[f'ed_id_mom{suffix}'] = np.int32([sum(df['ed.identifier'].str.contains('Mom_', regex=False))])
    res[f'ed_id_sid{suffix}'] = np.int32([sum(df['ed.identifier'].str.contains('sid_', regex=False))])
    positives = {'Dot_SoCool', 'Dot_GreatJob', 'ohWow', 'wowSoCool', 'thatLooksSoCool', 'tub_success', 
                 'water_success', 'soap_success', 'Dot_Amazing', 'Dot_WhoaSoCool', 'Dot_ThatsIt', 'youDidIt_1305',
                 'SFX_completedtask', 'Cleo_AmazingPowers', 'RIGHTANSWER1', 'Dot_Awesome', 'greatJob_1306', 'YouDidIt',
                 'RIGHTANSWER3', 'RIGHTANSWER2', 'INSTRCOMPLETE', 'AWESOME', 'WayToGoTeam', 'Dot_NiceWorkAllMatch',
                 'GreatFlying', 'WeDidItOneRoundLeft', 'Cleo_AweOfYourSkills', 'Dot_NiceWork'}
    n_pos = 0
    for p in positives:
        n_pos += sum(df['ed.identifier'].str.contains(p, regex=False))
    res[f'ed_id_positive{suffix}'] = np.int32([n_pos])
    negatives = {'Dot_Uhoh', 'Dot_UhOh', 'Dot_NeedTryAgain', 'IncorrectTooHeavy', 'Dot_GoLower', 'Buddy_TryDifferentNest',
                 'Cleo_BowlTooLight', 'Dot_GoHigher', 'Dot_SoLow', 'Dot_SoHigh', 'Dot_WhoopsTooShort', 'IncorrectTooLight',
                 'NOT_THAT_HEAVY', 'Dot_UhOhTooTall', 'ADD_MORE_WEIGHT', 'wrong1', 'tryAgain1', 'Dot_TryWeighingAgain',
                 'Cleo_RememberHeavierBowl', 'Dot_Whoops', 'Dot_NotBalanced', 'Mom_TooManyContainers',
                 'WrongOver', 'Mom_TooMuchWater', 'Dot_ThatBucketNotRight', 'Dot_TryAgain', 'wrongFewer', 'WrongBetweenCliff',
                 'Mom_NeedMoreContainers', 'Dot_Try', 'Dot_HmTooSmall'}
    n_neg = 1
    for ne in negatives:
        n_neg += sum(df['ed.identifier'].str.contains(ne, regex=False))
    res[f'ed_id_negative{suffix}'] = np.int32([n_neg])
    res[f'ed_id_positive_ratio{suffix}'] = np.float32([n_pos / n_neg])
    return res
    

def _features_map(df, EVENT_CODES, EVENT_IDS, TITLES, TYPES, WORLDS, suffix=''):
    res = {}
    for t in TYPES:
        res[f'type_{_key(t)}{suffix}'] = np.int32([sum(df['type'] == t)])
    
    for t in WORLDS:
        res[f'world_{_key(t)}{suffix}'] = np.int32([sum(df['world'] == t)])
    
    for t in TITLES:
        res[f'title_{_key(t)}{suffix}'] = np.int32([sum(df['title'] == t)])
    
    cols = ['game_time', 'event_count']
    for col in cols:
        res.update(_game_session_stats(df, col, suffix))
    
    res.update(_event_code_features(df, EVENT_CODES, TITLES=[], TYPES=[], WORLDS=[], suffix=suffix))
    # _event_id_features is slow!
    res.update(_event_id_features(df, EVENT_IDS, TITLES=[], TYPES=[], WORLDS=[], suffix=suffix))
    res.update(_event_data_features(df, suffix))
    return res


def _features(df, installation_id, EVENT_CODES, EVENT_IDS, TITLES, TYPES, WORLDS):
    res = {}
    iid = df[df['installation_id'] == installation_id].copy()
    if TARGET in df.columns:
        cutoff = _cutoff(iid, TARGET)
        iid = _trim_events_after_last_assessment(iid, cutoff)
        res[TARGET] = _target_variable(iid, cutoff, TARGET)
    res['installation_id'] = [installation_id]    
    res.update(_features_map(iid, EVENT_CODES, EVENT_IDS, TITLES, TYPES, WORLDS))
    return pd.DataFrame.from_dict(res)


def _preprocess(raw, EVENT_CODES, EVENT_IDS, TITLES, TYPES, WORLDS):
    res = pd.DataFrame()
    iids = raw['installation_id'].unique()
    for iid in tqdm(iids):
        tmp = _features(raw, iid, EVENT_CODES, EVENT_IDS, TITLES, TYPES, WORLDS)
        res = pd.concat([res, tmp], ignore_index=True)
    return res


train = _preprocess(train_raw, EVENT_CODES, EVENT_IDS, TITLES, TYPES, WORLDS)











  0%|                                                         | 0/3614 [00:00<?, ?it/s]









  0%|                                                 | 1/3614 [00:00<53:21,  1.13it/s]









  0%|                                                 | 2/3614 [00:01<52:42,  1.14it/s]









  0%|                                                 | 3/3614 [00:02<48:40,  1.24it/s]









  0%|                                                 | 4/3614 [00:03<46:26,  1.30it/s]









  0%|                                                 | 5/3614 [00:03<44:59,  1.34it/s]









  0%|                                                 | 6/3614 [00:04<47:37,  1.26it/s]









  0%|                                                 | 7/3614 [00:05<50:25,  1.19it/s]









  0%|                                                 | 8/3614 [00:06<49:46,  1.21it/s]









  0%|                                                 | 9/3614 [00:07<46:52,  1.28it/s]









  0%|▏              

  2%|▊                                               | 64/3614 [00:49<41:45,  1.42it/s]









  2%|▊                                               | 65/3614 [00:50<40:45,  1.45it/s]









  2%|▉                                               | 66/3614 [00:50<41:35,  1.42it/s]









  2%|▉                                               | 67/3614 [00:51<40:24,  1.46it/s]









  2%|▉                                               | 68/3614 [00:52<50:27,  1.17it/s]









  2%|▉                                               | 69/3614 [00:53<53:01,  1.11it/s]









  2%|▉                                               | 70/3614 [00:54<51:21,  1.15it/s]









  2%|▉                                               | 71/3614 [00:55<48:08,  1.23it/s]









  2%|▉                                               | 72/3614 [00:56<47:49,  1.23it/s]









  2%|▉                                               | 73/3614 [00:56<45:32,  1.30it/s]









  2%|▉                        

  4%|█▋                                             | 128/3614 [01:44<48:30,  1.20it/s]









  4%|█▋                                             | 129/3614 [01:44<46:04,  1.26it/s]









  4%|█▌                                           | 130/3614 [01:46<1:01:12,  1.05s/it]









  4%|█▋                                             | 131/3614 [01:47<54:16,  1.07it/s]









  4%|█▋                                             | 132/3614 [01:47<49:48,  1.17it/s]









  4%|█▋                                             | 133/3614 [01:48<48:49,  1.19it/s]









  4%|█▋                                             | 134/3614 [01:49<48:22,  1.20it/s]









  4%|█▊                                             | 135/3614 [01:50<46:21,  1.25it/s]









  4%|█▊                                             | 136/3614 [01:50<44:26,  1.30it/s]









  4%|█▊                                             | 137/3614 [01:51<43:22,  1.34it/s]









  4%|█▊                       

  5%|██▍                                            | 192/3614 [02:35<53:54,  1.06it/s]









  5%|██▌                                            | 193/3614 [02:36<51:20,  1.11it/s]









  5%|██▌                                            | 194/3614 [02:37<47:14,  1.21it/s]









  5%|██▌                                            | 195/3614 [02:38<51:27,  1.11it/s]









  5%|██▌                                            | 196/3614 [02:39<50:03,  1.14it/s]









  5%|██▌                                            | 197/3614 [02:39<47:35,  1.20it/s]









  5%|██▌                                            | 198/3614 [02:40<44:31,  1.28it/s]









  6%|██▌                                            | 199/3614 [02:41<42:51,  1.33it/s]









  6%|██▌                                            | 200/3614 [02:41<42:08,  1.35it/s]









  6%|██▌                                            | 201/3614 [02:42<44:21,  1.28it/s]









  6%|██▋                      

  7%|███▎                                           | 256/3614 [03:24<44:43,  1.25it/s]









  7%|███▎                                           | 257/3614 [03:25<42:35,  1.31it/s]









  7%|███▎                                           | 258/3614 [03:26<43:57,  1.27it/s]









  7%|███▎                                           | 259/3614 [03:27<41:30,  1.35it/s]









  7%|███▍                                           | 260/3614 [03:27<43:05,  1.30it/s]









  7%|███▍                                           | 261/3614 [03:28<41:54,  1.33it/s]









  7%|███▍                                           | 262/3614 [03:29<40:16,  1.39it/s]









  7%|███▍                                           | 263/3614 [03:30<47:03,  1.19it/s]









  7%|███▍                                           | 264/3614 [03:31<49:54,  1.12it/s]









  7%|███▍                                           | 265/3614 [03:32<50:07,  1.11it/s]









  7%|███▍                     

...

 92%|██████████████████████████████████████████▎   | 3328/3614 [45:01<05:46,  1.21s/it]









 92%|██████████████████████████████████████████▎   | 3329/3614 [45:01<04:56,  1.04s/it]









 92%|██████████████████████████████████████████▍   | 3330/3614 [45:02<04:24,  1.07it/s]









 92%|██████████████████████████████████████████▍   | 3331/3614 [45:03<04:00,  1.17it/s]









 92%|██████████████████████████████████████████▍   | 3332/3614 [45:03<03:58,  1.18it/s]









 92%|██████████████████████████████████████████▍   | 3333/3614 [45:04<03:58,  1.18it/s]









 92%|██████████████████████████████████████████▍   | 3334/3614 [45:05<03:40,  1.27it/s]









 92%|██████████████████████████████████████████▍   | 3335/3614 [45:06<03:30,  1.33it/s]









 92%|██████████████████████████████████████████▍   | 3336/3614 [45:06<03:24,  1.36it/s]









 92%|██████████████████████████████████████████▍   | 3337/3614 [45:07<03:16,  1.41it/s]









 92%|█████████████████████████

 94%|███████████████████████████████████████████▏  | 3392/3614 [45:51<03:14,  1.14it/s]









 94%|███████████████████████████████████████████▏  | 3393/3614 [45:52<03:00,  1.22it/s]









 94%|███████████████████████████████████████████▏  | 3394/3614 [45:53<04:06,  1.12s/it]









 94%|███████████████████████████████████████████▏  | 3395/3614 [45:54<03:42,  1.02s/it]









 94%|███████████████████████████████████████████▏  | 3396/3614 [45:55<03:22,  1.08it/s]









 94%|███████████████████████████████████████████▏  | 3397/3614 [45:56<03:10,  1.14it/s]









 94%|███████████████████████████████████████████▎  | 3398/3614 [45:56<02:56,  1.22it/s]









 94%|███████████████████████████████████████████▎  | 3399/3614 [45:57<02:47,  1.28it/s]









 94%|███████████████████████████████████████████▎  | 3400/3614 [45:58<02:39,  1.34it/s]









 94%|███████████████████████████████████████████▎  | 3401/3614 [45:58<02:34,  1.38it/s]









 94%|█████████████████████████

 96%|███████████████████████████████████████████▉  | 3456/3614 [46:44<01:58,  1.33it/s]









 96%|████████████████████████████████████████████  | 3457/3614 [46:45<01:53,  1.38it/s]









 96%|████████████████████████████████████████████  | 3458/3614 [46:46<01:52,  1.39it/s]









 96%|████████████████████████████████████████████  | 3459/3614 [46:46<01:56,  1.34it/s]









 96%|████████████████████████████████████████████  | 3460/3614 [46:47<02:03,  1.25it/s]









 96%|████████████████████████████████████████████  | 3461/3614 [46:48<02:17,  1.11it/s]









 96%|████████████████████████████████████████████  | 3462/3614 [46:49<02:09,  1.17it/s]









 96%|████████████████████████████████████████████  | 3463/3614 [46:50<01:59,  1.26it/s]









 96%|████████████████████████████████████████████  | 3464/3614 [46:51<02:04,  1.20it/s]









 96%|████████████████████████████████████████████  | 3465/3614 [46:51<01:54,  1.30it/s]









 96%|█████████████████████████

 97%|████████████████████████████████████████████▊ | 3520/3614 [47:39<01:07,  1.40it/s]









 97%|████████████████████████████████████████████▊ | 3521/3614 [47:39<01:04,  1.44it/s]









 97%|████████████████████████████████████████████▊ | 3522/3614 [47:40<01:12,  1.27it/s]









 97%|████████████████████████████████████████████▊ | 3523/3614 [47:42<01:28,  1.03it/s]









 98%|████████████████████████████████████████████▊ | 3524/3614 [47:43<01:20,  1.12it/s]









 98%|████████████████████████████████████████████▊ | 3525/3614 [47:43<01:12,  1.22it/s]









 98%|████████████████████████████████████████████▉ | 3526/3614 [47:44<01:07,  1.30it/s]









 98%|████████████████████████████████████████████▉ | 3527/3614 [47:45<01:09,  1.26it/s]









 98%|████████████████████████████████████████████▉ | 3528/3614 [47:46<01:24,  1.01it/s]









 98%|████████████████████████████████████████████▉ | 3529/3614 [47:47<01:16,  1.12it/s]









 98%|█████████████████████████

 99%|█████████████████████████████████████████████▌| 3584/3614 [48:31<00:22,  1.34it/s]









 99%|█████████████████████████████████████████████▋| 3585/3614 [48:32<00:22,  1.32it/s]









 99%|█████████████████████████████████████████████▋| 3586/3614 [48:33<00:21,  1.32it/s]









 99%|█████████████████████████████████████████████▋| 3587/3614 [48:33<00:19,  1.36it/s]









 99%|█████████████████████████████████████████████▋| 3588/3614 [48:34<00:18,  1.39it/s]









 99%|█████████████████████████████████████████████▋| 3589/3614 [48:35<00:17,  1.41it/s]









 99%|█████████████████████████████████████████████▋| 3590/3614 [48:36<00:17,  1.38it/s]









 99%|█████████████████████████████████████████████▋| 3591/3614 [48:37<00:18,  1.25it/s]









 99%|█████████████████████████████████████████████▋| 3592/3614 [48:37<00:16,  1.30it/s]









 99%|█████████████████████████████████████████████▋| 3593/3614 [48:38<00:15,  1.33it/s]









 99%|█████████████████████████

In [70]:
train.info(max_cols=9999)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3614 entries, 0 to 3613
Data columns (total 517 columns):
accuracy_group                       3614 non-null int64
installation_id                      3614 non-null object
type_clip                            3614 non-null int32
type_game                            3614 non-null int32
type_activity                        3614 non-null int32
type_assessment                      3614 non-null int32
world_none                           3614 non-null int32
world_magmapeak                      3614 non-null int32
world_crystalcaves                   3614 non-null int32
world_treetopcity                    3614 non-null int32
title_welcometolostlagoon            3614 non-null int32
title_magmapeaklevel1                3614 non-null int32
title_magmapeaklevel2                3614 non-null int32
title_crystalcaveslevel1             3614 non-null int32
title_chowtime                       3614 non-null int32
title_balancingact                  

In [ ]:
assert train.notna().all(axis=None)

In [71]:
train.head(10)

,accuracy_group,installation_id,type_clip,type_game,type_activity,type_assessment,world_none,world_magmapeak,world_crystalcaves,world_treetopcity,title_welcometolostlagoon,title_magmapeaklevel1,title_magmapeaklevel2,title_crystalcaveslevel1,title_chowtime,title_balancingact,title_crystalcaveslevel2,title_treetopcitylevel1,title_allstarsorting,title_costumebox,title_fireworksactivity,title_12monkeys,title_liftingheavythings,title_cartbalancerassessment,title_crystalcaveslevel3,title_eggdropperactivity,title_bugmeasureractivity,title_sandcastlebuilderactivity,title_dinodive,title_cauldronfillerassessment,title_scrubadub,title_wateringholeactivity,title_dinodrink,title_bubblebath,title_bottlefilleractivity,title_chickenbalanceractivity,title_honeycake,title_happycamel,title_leafleader,title_heavyheavierheaviest,title_panbalance,title_chestsorterassessment,title_orderingspheres,title_treetopcitylevel2,title_flowerwatereractivity,title_piratestale,title_mushroomsorterassessment,title_airshow,title_treasuremap,title_treetopcitylevel3,title_crystalsrule,title_rulers,title_birdmeasurerassessment,title_slopproblem,game_time_p50_game_session,game_time_max_game_session,game_time_std_game_session,game_time_iqr_game_session,game_time_p50_game_sessiontype,game_time_max_game_sessiontype,game_time_std_game_sessiontype,game_time_iqr_game_sessiontype,game_time_p50_game_sessionworld,game_time_max_game_sessionworld,game_time_std_game_sessionworld,game_time_iqr_game_sessionworld,game_time_p50_game_sessiontitle,game_time_max_game_sessiontitle,game_time_std_game_sessiontitle,game_time_iqr_game_sessiontitle,event_count_p50_game_session,event_count_max_game_session,event_count_std_game_session,event_count_iqr_game_session,event_count_p50_game_sessiontype,event_count_max_game_sessiontype,event_count_std_game_sessiontype,event_count_iqr_game_sessiontype,event_count_p50_game_sessionworld,event_count_max_game_sessionworld,event_count_std_game_sessionworld,event_count_iqr_game_sessionworld,event_count_p50_game_sessiontitle,event_count_max_game_sessiontitle,event_count_std_game_sessiontitle,event_count_iqr_game_sessiontitle,event_2000,event_2010,event_2020,event_2025,event_2030,event_2035,event_2040,event_2050,event_2060,event_2070,event_2075,event_2080,event_2081,event_2083,event_3010,event_3020,event_3021,event_3110,event_3120,event_3121,event_4010,event_4020,event_4021,event_4022,event_4025,event_4030,event_4031,event_4035,event_4040,event_4045,event_4050,event_4070,event_4080,event_4090,event_4095,event_4100,event_4110,event_4220,event_4230,event_4235,event_5000,event_5010,eid_0086365d,eid_00c73085,eid_022b4259,eid_02a42007,eid_0330ab6a,eid_0413e89d,eid_04df9b66,eid_05ad839b,eid_06372577,eid_070a5291,eid_08fd73f3,eid_08ff79ad,eid_0a08139c,eid_0d18d96c,eid_0d1da71f,eid_0db6d71d,eid_1325467d,eid_1340b8d7,eid_1375ccb7,eid_14de4c5d,eid_155f62a4,eid_1575e76c,eid_15a43e5b,eid_15ba1109,eid_15eb4a7d,eid_15f99afc,eid_160654fd,eid_16667cc5,eid_16dffff1,eid_17113b36,eid_19967db1,eid_1996c610,eid_1af8be29,eid_1bb5fbdb,eid_1beb320a,eid_1c178d24,eid_1cc7cfca,eid_1cf54632,eid_1f19558b,eid_222660ff,eid_2230fab4,eid_250513af,eid_25fa8af4,eid_262136f4,eid_26a5a3dd,eid_26fd2d99,eid_27253bdc,eid_28520915,eid_28a4eb9a,eid_28ed704e,eid_28f975ea,eid_29bdd9ba,eid_29f54413,eid_2a444e03,eid_2a512369,eid_2b058fe3,eid_2b9272f4,eid_2c4e6db0,eid_2dc29e21,eid_2dcad279,eid_2fb91ec1,eid_30614231,eid_30df3273,eid_31973d56,eid_3323d7e9,eid_33505eae,eid_3393b68b,eid_363c86c9,eid_363d3849,eid_36fa3ebe,eid_37937459,eid_37c53127,eid_37db1c2f,eid_37ee8496,eid_38074c54,eid_392e14df,eid_3a4be871,eid_3afb49e6,eid_3afde5dd,eid_3b2048ee,eid_3babcb9b,eid_3bb91ced,eid_3bb91dda,eid_3bf1cf26,eid_3bfd1a65,eid_3ccd3f02,eid_3d0b9317,eid_3d63345e,eid_3d8c61b0,eid_3dcdda7f,eid_3ddc79c3,eid_3dfd4aa4,eid_3edf6747,eid_3ee399c3,eid_44cb4907,eid_45d01abe,eid_461eace6,eid_46b50ba8,eid_46cd75b4,eid_47026d5f,eid_47efca07,eid_47f43a44,eid_48349b14,eid_4901243f,eid_499edb7c,eid_49ed92e9,eid_4a09ace1,eid_4a4c3d21,eid

In [ ]:
test = _preprocess(test_raw, EVENT_CODES, EVENT_IDS, TITLES, TYPES, WORLDS)











  0%|                                                         | 0/1000 [00:00<?, ?it/s]









  0%|                                                 | 1/1000 [00:00<05:10,  3.22it/s]









  0%|                                                 | 2/1000 [00:00<06:05,  2.73it/s]









  0%|▏                                                | 3/1000 [00:01<05:29,  3.02it/s]









  0%|▏                                                | 4/1000 [00:01<05:07,  3.24it/s]









  0%|▏                                                | 5/1000 [00:01<05:12,  3.19it/s]









  1%|▎                                                | 6/1000 [00:01<05:11,  3.20it/s]









  1%|▎                                                | 7/1000 [00:02<05:12,  3.18it/s]









  1%|▍                                                | 8/1000 [00:02<05:39,  2.92it/s]









  1%|▍                                                | 9/1000 [00:03<05:54,  2.79it/s]









  1%|▍              

  6%|███                                             | 64/1000 [00:23<05:31,  2.82it/s]









  6%|███                                             | 65/1000 [00:24<08:46,  1.78it/s]









  7%|███▏                                            | 66/1000 [00:24<07:29,  2.08it/s]









  7%|███▏                                            | 67/1000 [00:25<06:47,  2.29it/s]









  7%|███▎                                            | 68/1000 [00:25<05:58,  2.60it/s]









  7%|███▎                                            | 69/1000 [00:25<05:23,  2.87it/s]









  7%|███▎                                            | 70/1000 [00:26<05:04,  3.06it/s]









  7%|███▍                                            | 71/1000 [00:26<04:41,  3.30it/s]









  7%|███▍                                            | 72/1000 [00:26<04:42,  3.29it/s]









  7%|███▌                                            | 73/1000 [00:26<04:34,  3.38it/s]









  7%|███▌                     

 13%|██████                                         | 128/1000 [00:46<04:57,  2.94it/s]









 13%|██████                                         | 129/1000 [00:47<04:50,  3.00it/s]









 13%|██████                                         | 130/1000 [00:47<04:42,  3.08it/s]









 13%|██████▏                                        | 131/1000 [00:47<04:29,  3.22it/s]









 13%|██████▏                                        | 132/1000 [00:48<04:46,  3.03it/s]









 13%|██████▎                                        | 133/1000 [00:48<04:56,  2.92it/s]









 13%|██████▎                                        | 134/1000 [00:48<04:31,  3.19it/s]









 14%|██████▎                                        | 135/1000 [00:49<04:31,  3.18it/s]









 14%|██████▍                                        | 136/1000 [00:49<04:51,  2.97it/s]









 14%|██████▍                                        | 137/1000 [00:50<05:38,  2.55it/s]









 14%|██████▍                  

 19%|█████████                                      | 192/1000 [01:09<05:08,  2.62it/s]









 19%|█████████                                      | 193/1000 [01:10<04:46,  2.82it/s]









 19%|█████████                                      | 194/1000 [01:10<04:25,  3.04it/s]









 20%|█████████▏                                     | 195/1000 [01:10<04:04,  3.29it/s]









 20%|█████████▏                                     | 196/1000 [01:11<03:53,  3.44it/s]









 20%|█████████▎                                     | 197/1000 [01:11<03:47,  3.52it/s]









 20%|█████████▎                                     | 198/1000 [01:11<03:39,  3.66it/s]









 20%|█████████▎                                     | 199/1000 [01:12<04:21,  3.06it/s]









 20%|█████████▍                                     | 200/1000 [01:12<04:41,  2.84it/s]









 20%|█████████▍                                     | 201/1000 [01:12<04:21,  3.06it/s]









 20%|█████████▍               

 26%|████████████                                   | 256/1000 [01:32<03:47,  3.27it/s]









 26%|████████████                                   | 257/1000 [01:32<04:00,  3.09it/s]









 26%|████████████▏                                  | 258/1000 [01:32<03:49,  3.24it/s]









 26%|████████████▏                                  | 259/1000 [01:33<04:07,  2.99it/s]









 26%|████████████▏                                  | 260/1000 [01:33<04:31,  2.72it/s]









 26%|████████████▎                                  | 261/1000 [01:34<04:29,  2.74it/s]









 26%|████████████▎                                  | 262/1000 [01:34<04:09,  2.96it/s]









 26%|████████████▎                                  | 263/1000 [01:34<03:49,  3.20it/s]









 26%|████████████▍                                  | 264/1000 [01:34<03:40,  3.34it/s]









 26%|████████████▍                                  | 265/1000 [01:35<04:42,  2.60it/s]









 27%|████████████▌            

 32%|███████████████                                | 320/1000 [01:52<03:41,  3.07it/s]









 32%|███████████████                                | 321/1000 [01:52<03:26,  3.28it/s]









 32%|███████████████▏                               | 322/1000 [01:53<06:17,  1.80it/s]









 32%|███████████████▏                               | 323/1000 [01:53<05:15,  2.15it/s]









 32%|███████████████▏                               | 324/1000 [01:54<04:29,  2.51it/s]









 32%|███████████████▎                               | 325/1000 [01:54<03:57,  2.84it/s]









 33%|███████████████▎                               | 326/1000 [01:54<03:58,  2.83it/s]









 33%|███████████████▎                               | 327/1000 [01:55<05:51,  1.91it/s]









 33%|███████████████▍                               | 328/1000 [01:56<06:05,  1.84it/s]









 33%|███████████████▍                               | 329/1000 [01:56<05:05,  2.20it/s]









 33%|███████████████▌         

 38%|██████████████████                             | 384/1000 [02:15<02:56,  3.49it/s]









 38%|██████████████████                             | 385/1000 [02:15<02:49,  3.64it/s]









 39%|██████████████████▏                            | 386/1000 [02:16<02:57,  3.46it/s]









 39%|██████████████████▏                            | 387/1000 [02:16<02:47,  3.66it/s]









 39%|██████████████████▏                            | 388/1000 [02:16<02:54,  3.52it/s]









 39%|██████████████████▎                            | 389/1000 [02:17<02:56,  3.46it/s]









 39%|██████████████████▎                            | 390/1000 [02:17<02:52,  3.54it/s]









 39%|██████████████████▍                            | 391/1000 [02:17<02:56,  3.45it/s]









 39%|██████████████████▍                            | 392/1000 [02:17<02:47,  3.63it/s]









 39%|██████████████████▍                            | 393/1000 [02:18<02:47,  3.62it/s]









 39%|██████████████████▌      

 45%|█████████████████████                          | 448/1000 [02:37<03:58,  2.31it/s]









 45%|█████████████████████                          | 449/1000 [02:37<03:32,  2.59it/s]









 45%|█████████████████████▏                         | 450/1000 [02:37<03:26,  2.67it/s]









 45%|█████████████████████▏                         | 451/1000 [02:38<04:58,  1.84it/s]









 45%|█████████████████████▏                         | 452/1000 [02:39<04:15,  2.15it/s]









 45%|█████████████████████▎                         | 453/1000 [02:39<03:50,  2.37it/s]









 45%|█████████████████████▎                         | 454/1000 [02:39<03:21,  2.71it/s]









 46%|█████████████████████▍                         | 455/1000 [02:39<03:02,  2.99it/s]









 46%|█████████████████████▍                         | 456/1000 [02:40<02:52,  3.16it/s]









 46%|█████████████████████▍                         | 457/1000 [02:40<02:42,  3.34it/s]









 46%|█████████████████████▌   

 51%|████████████████████████                       | 512/1000 [03:02<03:40,  2.21it/s]









 51%|████████████████████████                       | 513/1000 [03:02<03:14,  2.50it/s]









 51%|████████████████████████▏                      | 514/1000 [03:03<02:52,  2.82it/s]









 52%|████████████████████████▏                      | 515/1000 [03:03<02:51,  2.83it/s]

In [ ]:
test.info(max_cols=9999)

In [ ]:
assert test.notna().all(axis=None)

In [ ]:
test.head(20)

In [ ]:
train.to_parquet('train.parquet')
test.to_parquet('test.parquet')
_log(os.listdir("."))

# Train Model

In [ ]:
%%time
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')

In [ ]:
def _log_transform(df, cols):
    df[cols] = np.float32(np.log(df[cols] + 1))


#cols = list(set(test.columns.values) - {'installation_id'})
#_log_transform(train, cols)
#_log_transform(test, cols)
#train.head()

In [ ]:
%%time
def _scaling(dfs, cols, scaler=None):
    scaler = sklearn.preprocessing.RobustScaler() if scaler is None else scaler
    scaler.fit(dfs[0][cols])
    for df in dfs:
        df[cols] = np.float32(scaler.transform(df[cols]))
        assert df.notna().all(axis=None)


#scaler = sklearn.preprocessing.PowerTransformer()
cols = list(set(test.columns.values) - {'installation_id'})
_scaling([train, test], cols)
train.head()

In [ ]:
def _keep_feature(left, right, alpha):
    return ks_2samp(left, right).pvalue > alpha


ALPHA = float('1e-04')
ps = set(test.columns.values) - {'installation_id'}
len_original = len(ps)
PREDICTORS = []
for p in tqdm(ps):
    if _keep_feature(train[p], test[p], ALPHA):
        PREDICTORS.append(p)
        
PREDICTORS = sorted(PREDICTORS)
_log(f'alpha={ALPHA}, keep {len(PREDICTORS)}/{len_original} features, drop {len_original - len(PREDICTORS)} features.\nPREDICTORS={PREDICTORS}')

In [ ]:
def _rmse(y, y_pred):
    return sqrt(mean_squared_error(y, y_pred))


SCORING = make_scorer(_rmse, greater_is_better = False)

In [ ]:
%%time
y_train = train[TARGET].astype(int)
x_train = train[PREDICTORS]
model = lgb.LGBMRegressor(n_estimators=ESTIMATORS, reg_alpha=1)
pipe = Pipeline([('model', model)])
param_grid = {
    'model__learning_rate': [0.01],
    'model__num_leaves': [16],
    'model__min_child_samples': [200],
    'model__colsample_bytree': [0.5]
}
cv = GridSearchCV(pipe, cv=FOLDS, param_grid=param_grid, scoring=SCORING)
#cv.fit(x_train, y_train, model__early_stopping_rounds=200, model__verbose=500)
cv.fit(x_train, y_train)
_log(f'best_params_={cv.best_params_}\nbest_score_={cv.best_score_:.5f}')

In [ ]:
# plot_metric only works with early stopping rounds
#lgb.plot_metric(cv.best_estimator_['model'])

In [ ]:
lgb.plot_importance(cv.best_estimator_['model'], max_num_features=100, figsize=(10, 30))

# Predict out of fold

In [ ]:
oof = train[['installation_id']].copy()
oof[TARGET] = cv.predict(x_train)
assert oof[TARGET].min() > -0.5
assert oof[TARGET].max() < 3.5
oof[TARGET] = np.round(oof[TARGET]).astype(int)
oof.head()

In [ ]:
score = cohen_kappa_score(oof[TARGET], y_train, weights='quadratic')
_log(f'oof score={score:.5f}')

# Predict on Test set

In [ ]:
x_test = test[PREDICTORS]
sub = test[['installation_id']].copy()
sub[TARGET] = cv.predict(x_test)
assert sub[TARGET].min() > -0.5
assert sub[TARGET].max() < 3.5
sub[TARGET] = np.round(sub[TARGET]).astype(int)
sub.head()

In [ ]:
plt.subplot(1, 3, 1)
plt.title('test predict')
sub[TARGET].plot(kind='hist')
plt.subplot(1, 3, 2)
plt.title('oof predict')
oof[TARGET].plot(kind='hist')
plt.subplot(1, 3, 3)
plt.title('oof truth')
tmp = train[TARGET].copy()
tmp = tmp.astype(int)
tmp.plot(kind='hist')
plt.tight_layout()

In [ ]:
sub.to_csv('submission.csv', index=False)
_log(os.listdir("."))